In [1]:
import jieba
#import synonyms
import random
from random import shuffle

random.seed(2022)
import numpy as np
import pandas as pd
from tqdm import tqdm

In [11]:
data_root = './data/'
corpus_data = pd.read_csv(data_root + "corpus.tsv", sep="\t", names=["doc", "title"])
dev_data = pd.read_csv(data_root + "dev.query.txt", sep="\t", names=["query", "title"])
train_data = pd.read_csv(data_root + "train.query.txt", sep="\t", names=["query", "title"])
qrels = pd.read_csv(data_root + "qrels.train.tsv", sep="\t", names=["query", "doc"])
corpus_data = corpus_data.set_index("doc")
dev_data = dev_data.set_index("query")
train_data = train_data.set_index("query")
qrels = qrels.set_index("query")

In [2]:
f = open('hit_stopwords.txt', encoding='utf-8')
stop_words = list()
for stop_word in f.readlines():
    stop_words.append(stop_word[:-1])

In [3]:
query_embeddings = np.load('./npy/query_embeddings.npy')
doc_embeddings = np.load('./npy/doc_embeddings.npy')
query_embeddings.shape, doc_embeddings.shape

((100000, 128), (1001500, 128))

In [4]:
sim = np.zeros((query_embeddings.shape[0], 5))
for i in tqdm(range(query_embeddings.shape[0])):
    sim[i] = np.dot(query_embeddings[i:i+1], doc_embeddings.T).argsort()[0,::-1][:5]
np.save('./npy/sim.npy', sim)
sim = sim.astype(np.int64)
sim = sim+1
neg_qrels = np.zeros((sim.shape[0],4))
for idx in tqdm(range(sim.shape[0])):
    pos_idx = qrels.iloc[idx].ravel()[0]
    neg_idx = sim[idx].tolist()
    if pos_idx in neg_idx:
        neg_idx.remove(pos_idx)
    else:
        neg_idx = neg_idx[:4]
    neg_qrels[idx] = np.array([neg_idx])
np.save('./npy/neg_qrels.npy', neg_qrels)

100%|██████████| 100000/100000 [5:00:15<00:00,  5.55it/s] 


In [56]:
corpus_data.loc[neg_qrels[1]]

,title
doc,
519689.0,手动304不锈钢绞肉机家用小型手摇碎肉料理机手动香肠灌肠机搅馅
398307.0,商用绞肉机商用手动灌肠机多功能装腊肠罐香肠机绞大蒜手摇绞菜机
472555.0,全自动电动切菜机碎菜机机器颗粒菜陷机养殖喂鸡刹菜机打绞菜商用
224520.0,厂家供应商用多功能丸子机小型肉丸机各种蔬菜罗卜自动丸子成型机


In [14]:
sentence = corpus_data['title'].loc[qrels.iloc[0].ravel()[0]]
words = list(jieba.cut(sentence))
words = [word for word in words if word not in stop_words]
num_words = len(words)

In [22]:
sentence, "".join(words)

('领券满减】美赞臣安婴儿A+亲舒 婴儿奶粉1段850克 0-12个月宝宝', '领券满减美赞臣安婴儿A亲舒 婴儿奶粉1段850克 012月宝宝')

In [33]:
augmented_sentences = []
num_aug = 2
alpha = 0.1
num_new_per_technique = int(num_aug/2)
n = max(1, int(alpha * num_words))
print(n)
#随机插入ri
for _ in range(num_new_per_technique):
    a_words = random_insertion(words, n)
    augmented_sentences.append(''.join(a_words))

1
满减


In [30]:
augmented_sentences

['领券满减美赞臣安婴儿A亲舒 婴儿奶粉1段850克 012月12宝宝']

In [32]:
import math
def is_number(s):
    if s != s.strip():
        return False
    try:
        f = float(s)
        if math.isnan(f) or math.isinf(f):
            return False
        return True
    except ValueError:
        return False

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words

def add_word(new_words):
    counter = 0
    while counter < 10:  
        random_word = new_words[random.randint(0, len(new_words)-1)]
        counter += 1
        if len(random_word) > 1 and not is_number(random_word):
            break
    random_idx = random.randint(0, len(new_words)-1)
    print(random_word)
    new_words.insert(random_idx, random_word)

def eda(sentence, alpha, p_rd=0.1, num_aug=2):
    seg_list = jieba.cut(sentence)
    seg_list = " ".join(seg_list)
    words = list(seg_list.split())
    words = [word for word in words if word not in stop_words]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/4)+1
    n = max(1, int(alpha * num_words))

    #随机插入ri
    for _ in range(num_new_per_technique):
        a_words = random_insertion(words, n)
        augmented_sentences.append(' '.join(a_words))
    
    #随机交换rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(' '.join(a_words))

   
    #随机删除rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentences.append(' '.join(a_words))
    
    #print(augmented_sentences)
    shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    augmented_sentences.append(seg_list)

    return augmented_sentences

In [ ]:
print(sim.shape)
np.save('./npy/sim.npy', sim)